In [ ]:
# Import required Julia packages for oceanographic data analysis
using NCDatasets, PhysOcean, DataStructures    # NetCDF I/O, oceanography utilities, data structures
using DIVAnd, PyPlot, Dates                     # DIVA interpolation, plotting, date handling
using Statistics, Random, Printf                # Statistical functions, random numbers, string formatting

#load dataset
datafile = "./nc/nitrogen.nc"

In [ ]:
# Define spatial grid parameters for the Mediterranean Sea analysis
# CORRECTED: Optimized grid resolution for phosphorus analysis
# Standard resolution for nutrient analysis - phosphorus has less spatial variability than chlorophyll-a
dx, dy = 0.05, 0.05          # Grid resolution in degrees (longitude, latitude) - standard for nutrients
lonr = -6:dx:37            # Longitude range from -6° to 37° E covering entire Mediterranean
latr = 30:dy:46            # Latitude range from 30° to 46° N covering entire Mediterranean
timerange = [Date(2003,06,06),Date(2012,01,01)];  # Time period for analysis

In [ ]:
# Define depth levels for dissolved inorganic nitrogen (DIN) 3D analysis (in meters)
# CORRECTED: Optimized depth levels for DIN distribution
# DIN shows vertical gradients throughout the water column with depletion in surface waters
# and accumulation at depth due to remineralization processes

# Optimized depth levels for DIN analysis:
# Standard nutrient sampling depths with emphasis on surface and intermediate waters
# DIN is crucial for primary production in surface waters and shows strong vertical gradients
depthr = [0., 5., 10., 20., 30., 40., 50., 75., 100., 125., 150., 200., 300., 400., 500., 750., 1000.];  # Extended depth range

# Define analysis parameters
varname = "Aggregated Water body dissolved inorganic nitrogen (DIN)"    # CORRECTED: Using actual variable name from dataset
yearlist = [2003:2012]; # Years to include in analysis

# CORRECTED: Seasonal groupings following EMODnet Chemistry guidelines (Page 35)
# Mediterranean seasons: winter (Jan-Mar), spring (Apr-Jun), summer (Jul-Sep), autumn (Oct-Dec)
monthlist = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]; # Winter, Spring, Summer, Autumn - EMODnet standard

# Create time selector for seasonal analysis
TS = DIVAnd.TimeSelectorYearListMonthList(yearlist,monthlist);
@show TS;

In [ ]:
# Then load from full dataset (overwrites the small dataset variables)
# Use the correct long_name attribute: "Aggregated Water body dissolved inorganic nitrogen (DIN)"
@time obsval,obslon,obslat,obsdepth,obstime,obsid = NCODV.load(Float64, datafile, 
    "Aggregated Water body dissolved inorganic nitrogen (DIN)");

# ========================================================================
# PLOTTING OBSERVATIONAL DATA DISTRIBUTION
# ========================================================================

# Create a figure showing the geographic distribution of observation points
figure("Mediterranean-Data")
ax = subplot(1,1,1)
plot(obslon, obslat, "ko", markersize=.1)  # Plot observation locations as small black dots
aspectratio = 1/cos(mean(latr) * pi/180)   # Calculate proper aspect ratio for latitude
ax.tick_params("both",labelsize=6)
gca().set_aspect(aspectratio)
title("Mediterranean Sea DIN Observation Locations")

# Check quality and consistency of observations
checkobs((obslon,obslat,obsdepth,obstime),obsval,obsid)

In [ ]:
# Download bathymetry data (seafloor depth) for the Mediterranean Sea region
bathname = "./nc/gebco_30sec_4.nc"

# Load bathymetry data and interpolate to our Mediterranean grid
@time bx,by,b = load_bath(bathname,true,lonr,latr);

# Plot the bathymetry data for the Mediterranean Sea
figure("Mediterranean-Bathymetry")
ax = subplot(1,1,1)
pcolor(bx, by, permutedims(b, [2,1]));  
colorbar(orientation="vertical", shrink=0.8, label="Depth (m)").ax.tick_params(labelsize=8)
contour(bx, by, permutedims(b, [2,1]), [0], colors="k", linewidths=1.0)  # Coastline
gca().set_aspect(aspectratio)
ax.tick_params("both",labelsize=6)
title("Mediterranean Sea Bathymetry")

# Create simple 3D mask - just water vs land based on bathymetry
mask = falses(size(b,1), size(b,2), length(depthr))

for k = 1:length(depthr)
    for j = 1:size(b,2)
        for i = 1:size(b,1)
            mask[i,j,k] = b[i,j] >= depthr[k]  # True where water depth >= analysis depth
        end
    end
end

# Plot the mask (surface level)
figure("Mediterranean-Mask")
ax = subplot(1,1,1)
gca().set_aspect(aspectratio)
ax.tick_params("both",labelsize=6)
pcolor(bx,by, transpose(mask[:,:,1])); 
title("Mediterranean Sea Water Mask")

In [ ]:
# ========================================================================
# DIVAND ANALYSIS PARAMETERS SETUP (EMODnet Chemistry Standards)
# ========================================================================

# Following EMODnet Chemistry DIVA Guidelines (Page 37-38)
# "EMODnet Chemistry group agreed on the use of fixed L and SN for all DIVA runs"
# Parameters should be obtained by estimation from a good subsample

# Optional: Calculate observation weights based on data density
# Recommended for high-density datasets to account for spatial clustering
@time rdiag=1.0./DIVAnd.weight_RtimesOne((obslon,obslat),(0.05,0.05));
@show maximum(rdiag),mean(rdiag)

# Define grid dimensions for parameter arrays
sz = (length(lonr),length(latr),length(depthr));

# Set correlation lengths (influence radius) for each dimension
# CORRECTED: Following EMODnet DIVA guidelines for Mediterranean DIN
# Based on EMODnet recommendation: "Minimal L (larger than output grid spacing): 0.25, Maximal L: 10"
# Grid resolution is 0.1° ≈ 11 km, so minimum correlation length should be ~22 km

# For DIN in Mediterranean (smoother spatial distribution similar to phosphorus):
lenx = fill(75_000.,sz)    # 75 km correlation length in longitude direction (nutrient spatial coherence)
leny = fill(75_000.,sz)    # 75 km correlation length in latitude direction (nutrient spatial coherence)
lenz = fill(40.,sz);       # 40 m correlation length in depth direction (DIN vertical gradients)
len = (lenx, leny, lenz);  # Combine into tuple for DIVAnd

# Set noise-to-signal ratio (regularization parameter)
# CORRECTED: Following EMODnet guidelines "Minimal SN: 0.1, Maximal SN: 3"
# Moderate epsilon2 for DIN - good measurement accuracy but some natural variability
epsilon2 = 0.08;           # Within EMODnet recommended range for nutrient parameters
epsilon2 = epsilon2 * rdiag;  # Apply spatially varying epsilon based on data density

In [ ]:
# ========================================================================
# OUTPUT FILE SETUP AND METADATA CONFIGURATION
# ========================================================================

# Set up output directory and filename
outputdir = "./"
if !isdir(outputdir)
    mkpath(outputdir)
end
filename = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_"))_Mediterranean.4Danl.nc")

# Define comprehensive metadata for NetCDF file following SeaDataNet standards
metadata = OrderedDict(
    # Name of the project (SeaDataCloud, SeaDataNet, EMODNET-chemistry, ...)
    "project" => "SeaDataCloud",

    # URN code for the institution EDMO registry,
    # e.g. SDN:EDMO::1579
    "institution_urn" => "SDN:EDMO::1579",

    # Production group
    #"production" => "Diva group",

    # Name and emails from authors
    "Author_e-mail" => ["Your Name1 <name1@example.com>", "Other Name <name2@example.com>"],

    # Source of the observation
    "source" => "observational data from SeaDataNet and World Ocean Atlas",

    # Additional comment
    "comment" => "Duplicate removal applied to the merged dataset. EMODnet Chemistry QC procedures applied.",

    # SeaDataNet Vocabulary P35 URN for dissolved inorganic nitrogen
    # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=p35
    "parameter_keyword_urn" => "SDN:P35::EPC00019", # Dissolved inorganic nitrogen concentration

    # List of SeaDataNet Parameter Discovery Vocabulary P02 URNs for nitrogen
    # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=p02
    "search_keywords_urn" => ["SDN:P02::NTRA"], # Nitrogen compounds concentrations

    # List of SeaDataNet Vocabulary C19 area URNs
    # SeaVoX salt and fresh water body gazetteer (C19)
    # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=C19
    "area_keywords_urn" => ["SDN:C19::3_1"], # Mediterranean Sea

    "product_version" => "1.0",
    
    "product_code" => "Mediterranean-DIN-Analysis",
    
    # bathymetry source acknowledgement
    "bathymetry_source" => "The GEBCO Digital Atlas published by the British Oceanographic Data Centre on behalf of IOC and IHO, 2003",

    # NetCDF CF standard name for dissolved inorganic nitrogen
    # http://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html
    "netcdf_standard_name" => "mole_concentration_of_dissolved_inorganic_nitrogen_in_sea_water",

    "netcdf_long_name" => "Mole concentration of dissolved inorganic nitrogen in sea water",

    "netcdf_units" => "umol l-1",

    # Abstract for the product
    "abstract" => "4D analysis of dissolved inorganic nitrogen (DIN) concentration in Mediterranean Sea using DIVAnd interpolation following EMODnet Chemistry methodology",

    # This option provides a place to acknowledge various types of support for the
    # project that produced the data
    "acknowledgement" => "EMODnet Chemistry project, SeaDataNet infrastructure",

    "documentation" => "https://doi.org/10.6092/9f75ad8a-ca32-4a72-bf69-167119b2cc12",

    # Digital Object Identifier of the data product
    "doi" => "...");

# Convert metadata to NetCDF-compatible attributes
ncglobalattrib, ncvarattrib = SDNMetadata(metadata, filename, varname, lonr, latr)

# Remove any existing analysis file to start fresh
if isfile(filename)
    rm(filename) # delete the previous analysis
    @info "Removing file $filename"
end

In [ ]:
# ========================================================================
# MAIN DIVAND ANALYSIS EXECUTION (OPTIMIZED FOR DIN)
# ========================================================================

# Execute the main DIVAnd 3D analysis
@time dbinfo = diva3d((lonr,latr,depthr,TS),        # Grid coordinates and time selector
    (obslon,obslat,obsdepth,obstime), obsval,        # Observation coordinates and values
    len, epsilon2,                                    # Correlation lengths and regularization
    filename,varname,                                 # Output file and variable name
    bathname=bathname,                               # Bathymetry file for land/sea mask
    #plotres = plotres,                               # CORRECTED: Enable plotting function for visualization
    mask = mask,                                # Edited mask for analysis domain
    fitcorrlen = false,                              # Don't fit correlation lengths automatically
    niter_e = 1,                                     # CORRECTED: Reduce iterations for faster computation
    ncvarattrib = ncvarattrib,                       # NetCDF variable attributes
    ncglobalattrib = ncglobalattrib,                 # NetCDF global attributes
    surfextend = true,                               # Extend surface values to deeper levels if needed
    memtofit = 3,                                    # CORRECTED: Optimize memory usage for large grids
    );

# Save observation metadata to the output file
DIVAnd.saveobs(filename,(obslon,obslat,obsdepth,obstime),obsid);